In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-100kv2/yelp-100kv2.csv


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

In [3]:
from huggingface_hub import login

login(secret_value_0)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [9]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2024-05-06 19:43:22.169262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 19:43:22.169395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 19:43:22.356386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Arrrr, shiver me timbers! Me name be Captain Chatbot, the scurviest chatbot to ever sail the Seven Seas! Me be a swashbucklin' bot, here to chat with ye about all manner o' things, from the latest booty to the best ways to avoid walkin' the plank! So hoist the colors, me hearty, and let's set sail fer a treasure trove o' conversation!


## DF

In [10]:
df = pd.read_csv('/kaggle/input/yelp-100kv2/yelp-100kv2.csv')
df.head()

,business_id,stars,text,name,categories
0,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,7ATYjTIgM3jUlt4UM3IypQ,5.0,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,YjUWPpI6HXG530lwP-fb2A,3.0,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,"Restaurants, Breakfast & Brunch"
3,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",Zaika,"Halal, Pakistani, Restaurants, Indian"
4,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...,Melt,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ..."


In [11]:
df['text'] = df['text'].replace('\n', '', regex=True)
df.rename(columns={'text': 'reviewText'}, inplace=True)
df.drop('business_id', axis=1, inplace=True)
df['stars']=df['stars'].astype(int)
df.head()

,stars,reviewText,name,categories
0,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,"Restaurants, Breakfast & Brunch"
3,5,"Wow! Yummy, different, delicious. Our favo...",Zaika,"Halal, Pakistani, Restaurants, Indian"
4,4,Cute interior and owner (?) gave us tour of up...,Melt,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ..."


In [12]:
df['Wygenerowane'] = ''

In [15]:
import time
from tqdm import tqdm

start_time = time.time()

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


range_of_numbers = tqdm(range(50011, 50501))  # Replace with your desired range of numbers
for nr in range_of_numbers:
    if isinstance(df['reviewText'][nr], str):
        min_length = 0 if len(df['reviewText'][nr])-20 < 0 else len(df['reviewText'][nr])-20
        max_length = len(df['reviewText'][nr])+20
    else:
        min_length = 0
        max_length = 20

    prompt6 = f"""Write a review for a visited restaurant. Ensure your review reflects a rating of {df['stars'][nr]} stars and the review length is between {min_length} and {max_length} characters."""

    messages = [
        {"role": "user", "content": prompt6},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_length/4,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    response = outputs[0][input_ids.shape[-1]:]
    review_text = tokenizer.decode(response, skip_special_tokens=True)

    #print(str(nr)+str(review_text))

    df.loc[nr, 'Wygenerowane'] = review_text

    if nr % 50 == 0:
        print(f'Processed {nr} record')
        execution_time = time.time() - start_time
        minutes = execution_time // 60
        seconds = execution_time % 60
        print(str(nr)+str(review_text))
        print(f"Czas wykonania: {int(minutes)} minut {seconds} sekund")

  0%|          | 0/490 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 1/490 [00:03<26:42,  3.28s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 2/490 [00:07<30:21,  3.73s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|          | 3/490 [00:11<32:30,  4.00s/it]The attention mask and the pad token id were not set. As a conse

Processed 50050 record
50050"I visited Bistro Bliss expecting a delightful dining experience, but unfortunately, it fell short. The service was slow and unfriendly, with long waits between courses. The food was decent, but nothing special. The ambiance was lacking, with dim lighting and uncomfortable seating. Overall, I'd give Bistro Bliss 2 stars. While it's not a complete disaster, it's not worth the hype. Maybe with some improvements, it could be
Czas wykonania: 4 minut 45.270782470703125 sekund


  8%|▊         | 41/490 [04:49<41:05,  5.49s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  9%|▊         | 42/490 [04:53<37:27,  5.02s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  9%|▉         | 43/490 [05:00<42:11,  5.66s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  9%|▉         | 44/490 [05:08<46:40,  6.28s/it]The attention mask and the pad token id were not set

Processed 50100 record
50100"I recently visited Bistro Bliss and was thoroughly impressed! The atmosphere was cozy and intimate, with a lovely patio area perfect for a romantic evening. The service was top-notch, with friendly and attentive staff. The menu offered a variety of creative dishes, and I opted for the seafood risotto which was cooked to perfection. The only reason I didn't give it 5 stars is that the portion sizes were a bit small. Overall, a wonderful dining experience and I would highly recommend it! 4/5 stars"
Czas wykonania: 9 minut 53.380157709121704 sekund


 19%|█▊        | 91/490 [09:58<38:44,  5.83s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 19%|█▉        | 92/490 [10:02<34:53,  5.26s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 19%|█▉        | 93/490 [10:12<42:46,  6.46s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 19%|█▉        | 94/490 [10:29<1:04:48,  9.82s/it]The attention mask and the pad token id were not s

...

Processed 50400 record
50400"Disappointing experience at Bistro Bliss. Food was overpriced and underseasoned, service was slow and unfriendly. Ambiance was nice, but not
Czas wykonania: 45 minut 36.42454504966736 sekund


 80%|███████▉  | 391/490 [45:44<11:39,  7.06s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 80%|████████  | 392/490 [45:53<12:20,  7.56s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 80%|████████  | 393/490 [46:08<15:55,  9.85s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 80%|████████  | 394/490 [46:14<13:50,  8.65s/it]The attention mask and the pad token id were not

Processed 50450 record
50450"I recently had the pleasure of dining at Bistro Bliss and was thoroughly impressed! The atmosphere was cozy and intimate, with a warm and inviting ambiance. The service was top-notch, with friendly and attentive staff who made sure our every need was met. The menu was diverse and creative, with dishes that were both flavorful and visually stunning. The standout dish for me was the pan-seared salmon, which was cooked to perfection and melted in my mouth. Overall, I would highly recommend Bistro Bliss to anyone looking for a
Czas wykonania: 52 minut 13.202617883682251 sekund


 90%|█████████ | 441/490 [52:16<05:25,  6.63s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 90%|█████████ | 442/490 [52:25<05:51,  7.31s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 90%|█████████ | 443/490 [52:31<05:24,  6.91s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 91%|█████████ | 444/490 [52:40<05:47,  7.56s/it]The attention mask and the pad token id were not

Processed 50500 record
50500"Delightful dinner at Bistro Bliss! The service was top-notch, and the menu offered a great variety of options. The seafood paella was cooked to
Czas wykonania: 57 minut 54.381909132003784 sekund


In [40]:
print(messages)

[{'role': 'system', 'content': 'Your task is to create a review that could easily be mistaken for a genuine user-written review. The review should integrate elements of style, tone, and linguistic nuances found in real consumer testimonials to ensure it blends seamlessly with legitimate content.'}, {'role': 'user', 'content': '\n    Write a review for a visited business. The review should be crafted to appear authentic, using a human-like language that mimics a genuine customer testimonial.\n    Use to the provided business details for context. Consider the example review as reference how a similar review looks like.\n    Ensure your review reflects a rating of 5 stars and the review length is between 445 and 485 characters.\n\n    Example Review for Reference:\n    "This is the best Starbucks in the city. I don\'t know what it is about this location (perhaps the fact that they only have a cafe and no drive thru) but the service here is always top notch. No waiting for 20 minutes for a

In [16]:
df.to_pickle('/kaggle/working/llama_checkpoint.pkl')

In [17]:
reviews_generated6_full = df.reset_index()
reviews_generated6_full['Wygenerowane'] = reviews_generated6_full['Wygenerowane'].replace('', np.nan)
reviews_generated6_full['text_'] = np.where(reviews_generated6_full['Wygenerowane'].notnull(), reviews_generated6_full['Wygenerowane'], reviews_generated6_full['reviewText'])
reviews_generated6_full['label'] = np.where(reviews_generated6_full['Wygenerowane'].notnull(), 'CG', 'OG')
reviews_generated6_full['target'] = np.where(reviews_generated6_full['Wygenerowane'].notnull(), 1, 0)

#[['index','reviewText', 'Wygenerowane']]
print(reviews_generated6_full)
reviews_generated6_full.to_excel('/kaggle/working/reviews_generated6_full.xlsx', index=True)
reviews_generated6_full.to_csv('/kaggle/working/reviews_generated6_full.csv', index=True)

       index  stars                                         reviewText  \
0          0      3  If you decide to eat here, just be aware it is...   
1          1      5  I've taken a lot of spin classes over the year...   
2          2      3  Family diner. Had the buffet. Eclectic assortm...   
3          3      5  Wow!  Yummy, different,  delicious.   Our favo...   
4          4      4  Cute interior and owner (?) gave us tour of up...   
...      ...    ...                                                ...   
99995  99995      5  Came in for my 5-6 month prophy and saw Kara -...   
99996  99996      3  Why 3 stars?I love Target, but here is just so...   
99997  99997      4  The Local Taco is the first joint I've found i...   
99998  99998      5  Top notch! He will cater our wedding and  Fran...   
99999  99999      5  Best place to get a vietnamese sandwich and bo...   

                                   name  \
0          Turning Point of North Wales   
1            Body Cycle S

In [19]:
reviews_generated6_cut = reviews_generated6_full[['text_', 'label', 'target']].iloc[50000:51000]
reviews_generated6_cut.columns = ['text', 'label', 'target']
reviews_generated6_cut.to_csv('/kaggle/working/reviews_generated6_cut_final.csv', index=True)
reviews_generated6_cut.to_excel('/kaggle/working/reviews_generated6_cut.xlsx', index=True)
print(reviews_generated6_cut)

                                                    text label  target
50000  Best corn ever! Taste great and always fresh f...    OG       0
50001  "I'm still trying to process the disaster that...    CG       1
50002  "I recently visited Bistro Bliss and was blown...    CG       1
50003  "I recently visited Bistro Bliss and had a del...    CG       1
50004  "I recently had the pleasure of dining at Bist...    CG       1
...                                                  ...   ...     ...
50995  After a heavy lunch at Monell's, my friends an...    OG       0
50996  Best milkshakes and burgers in town! Love the ...    OG       0
50997  I brought my family to Newks on Saturday 3/7/1...    OG       0
50998  This place is the pits.  They always forget so...    OG       0
50999  I have had authentic tacos throughout Central ...    OG       0

[1000 rows x 3 columns]
